<a href="https://colab.research.google.com/github/thinhngo-x/Anonymization/blob/master/notebook/Anonymization_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib.request import urlopen
! wget -nc "https://raw.githubusercontent.com/glample/tagger/master/dataset/eng.testa"
! wget -nc "https://raw.githubusercontent.com/glample/tagger/master/dataset/eng.train"
! wget -nc "https://raw.githubusercontent.com/glample/tagger/master/dataset/eng.testb"
# html = urlopen(url)
# with open('eng.testb', 'b') as the_file:
#     for line in html:
#         the_file.write(line.decode('UTF-8'))

File ‘eng.testa’ already there; not retrieving.

File ‘eng.train’ already there; not retrieving.

File ‘eng.testb’ already there; not retrieving.



In [2]:
# WNUT17 Dataset https://github.com/leondz/emerging_entities_17

! wget -nc "https://raw.githubusercontent.com/leondz/emerging_entities_17/master/wnut17train.conll"
! wget -nc "https://raw.githubusercontent.com/leondz/emerging_entities_17/master/emerging.test.conll"

File ‘wnut17train.conll’ already there; not retrieving.

File ‘emerging.test.conll’ already there; not retrieving.



# Projet informatique 9

## GDPR in practice: data anonymization

This quick notebook will show you how to anonymize text data automatically. You'll have to do the same, by testing other approaches and / or embeddings and / or classification algorithms.

### Some dependencies and some data

In [56]:
from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/My Drive/Anonymization/CONLL2003_BERT/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
! pip install transformers

In [0]:
import transformers
import os
from torch.nn import CrossEntropyLoss
import logging
logger = logging.getLogger(__name__)

### Some useful functions

In [0]:
class InputExample(object):
    """A single training/test example for token classification."""

    def __init__(self, guid, words, labels):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            words: list. The words of the sequence.
            labels: (Optional) list. The labels for each word of the sequence. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.words = words
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids


def read_examples_from_file(data_dir, mode="eng.testa"):
    file_path = os.path.join(data_dir, "{}".format(mode))
    guid_index = 1
    examples = []
    with open(file_path, encoding="utf-8") as f:
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n" or line == "\t\n":
                if words:
                    examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
                    guid_index += 1
                    words = []
                    labels = []
            else:
                splits = line.split("\t")
                print(splits)
                words.append(splits[0])
                if len(splits) > 1:
                    # print(splits[-1])
                    label = splits[-1].split(",")
                    if "B-person" in label:
                        labels.append("B-person")
                    elif "I-person" in label:
                        labels.append("I-person")
                    else:
                        labels.append(label[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
        if words:
            examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
    return examples


def convert_examples_to_features(
    examples,
    label_list,
    max_seq_length,
    tokenizer,
    cls_token_at_end=False,
    cls_token="[CLS]",
    cls_token_segment_id=1,
    sep_token="[SEP]",
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=0,
    pad_token_segment_id=0,
    pad_token_label_id=-100,
    sequence_a_segment_id=0,
    mask_padding_with_zero=True,
):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d", ex_index, len(examples))

        tokens = []
        label_ids = []
        for word, label in zip(example.words, example.labels):
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            # Use the real label id for the first token of the word, and padding ids for the remaining tokens
            print(label, pad_token_label_id, word_tokens)
            label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 3 if sep_token_extra else 2
        if len(tokens) > max_seq_length - special_tokens_count:
            tokens = tokens[: (max_seq_length - special_tokens_count)]
            label_ids = label_ids[: (max_seq_length - special_tokens_count)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids:   0   0   0   0  0     0   0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens += [sep_token]
        label_ids += [pad_token_label_id]
        if sep_token_extra:
            # roberta uses an extra separator b/w pairs of sentences
            tokens += [sep_token]
            label_ids += [pad_token_label_id]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if cls_token_at_end:
            tokens += [cls_token]
            label_ids += [pad_token_label_id]
            segment_ids += [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            label_ids = [pad_token_label_id] + label_ids
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            label_ids = ([pad_token_label_id] * padding_length) + label_ids
        else:
            input_ids += [pad_token] * padding_length
            input_mask += [0 if mask_padding_with_zero else 1] * padding_length
            segment_ids += [pad_token_segment_id] * padding_length
            label_ids += [pad_token_label_id] * padding_length

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s", example.guid)
            logger.info("tokens: %s", " ".join([str(x) for x in tokens]))
            logger.info("input_ids: %s", " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s", " ".join([str(x) for x in input_mask]))
            logger.info("segment_ids: %s", " ".join([str(x) for x in segment_ids]))
            logger.info("label_ids: %s", " ".join([str(x) for x in label_ids]))

        features.append(
            InputFeatures(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_ids=label_ids)
        )
    return features

### A direct, pre-trained NER approach

In [0]:
nlp = transformers.pipeline("ner")

In [0]:
nlp("Adrien Ehrhardt donne un projet d'informatique aux étudiants de INF442.")

[{'entity': 'I-PER', 'score': 0.9998049736022949, 'word': 'Ad'},
 {'entity': 'I-PER', 'score': 0.9996461272239685, 'word': '##rien'},
 {'entity': 'I-PER', 'score': 0.9998077154159546, 'word': 'E'},
 {'entity': 'I-PER', 'score': 0.992250919342041, 'word': '##hr'},
 {'entity': 'I-PER', 'score': 0.9994708299636841, 'word': '##hardt'},
 {'entity': 'I-ORG', 'score': 0.8828932642936707, 'word': 'IN'},
 {'entity': 'I-ORG', 'score': 0.5023669600486755, 'word': '##F'}]

### Obtaining the representation of the dataset from Bert

#### An example

In [0]:
from transformers import BertTokenizer, BertModel
import torch

In [0]:
model = BertModel.from_pretrained("bert-large-cased").cuda()
tokenizer = BertTokenizer.from_pretrained("bert-large-cased")

sequence = "Adrien Ehrhardt donne un projet d'informatique aux étudiants de INF442."
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")

outputs = model(inputs.cuda())[0]

In [0]:
outputs.detach().cpu().numpy()[0].shape

(28, 1024)

In [0]:
outputs

tensor([[[-0.3408, -0.5467,  0.3817,  ..., -0.3688, -0.3334,  0.2856],
         [-0.5390, -0.9849, -0.2608,  ..., -0.4244,  0.1998, -0.3972],
         [-0.5249,  0.2965,  0.0322,  ...,  0.8137,  0.4162, -0.4437],
         ...,
         [-0.6153,  0.5873,  0.0034,  ...,  0.0931,  0.1486,  0.5473],
         [-0.5728, -0.4093, -0.3757,  ...,  0.3390,  0.0722, -0.3506],
         [-0.7437, -0.5303, -0.8375,  ..., -0.5345,  0.6854, -0.2059]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward>)

#### The real thing

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

In [0]:
from tqdm.notebook import tqdm, trange

In [0]:
import numpy as np

In [42]:
pad_token_label_id=CrossEntropyLoss().ignore_index
labels = ["O", "B-location", "I-location", "B-person", "I-person", "B-group", "I-group",
          "B-corporation", "I-corporation", "B-product", "I-product",
          "B-creative-work", "I-creative-work"]
model = BertModel.from_pretrained("bert-large-cased").cuda()
tokenizer = BertTokenizer.from_pretrained("bert-large-cased")
examples = read_examples_from_file(".", mode="emerging.test.conll")
print(examples[2].labels)
features = convert_examples_to_features(
            examples,
            label_list = labels,
            max_seq_length = 128,
            tokenizer = tokenizer,
            cls_token_at_end=False,
            cls_token=tokenizer.cls_token,
            cls_token_segment_id=0,
            sep_token=tokenizer.sep_token,
            sep_token_extra=False,
            pad_on_left=False,
            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            pad_token_segment_id=0,
            pad_token_label_id=pad_token_label_id)

Streaming output truncated to the last 5000 lines.
O -100 ['_']
O -100 ['Mu', '##lt', '##ling']
O -100 ['R', '##T']
O -100 ['@']
O -100 ['a', '##p', '##ren', '##dering', '##les', '##z']
O -100 [':']
O -100 ['Life']
O -100 ['is']
O -100 ['meant']
O -100 ['to']
O -100 ['be']
O -100 ['fun']
O -100 [':']
O -100 ['A']
O -100 ['v', '##ida']
O -100 ['par', '##ec', '##e']
O -100 ['se', '##r']
O -100 ['diver', '##ti', '##da']
O -100 ['.']
O -100 ['R', '##T']
O -100 ['@']
O -100 ['female', '##book']
O -100 [':']
O -100 ['@']
O -100 ['my']
O -100 ['crush']
O -100 ['https', ':', '/', '/', 't', '.', 'co', '/', 'r', '##i', '##U', '##z', '##x', '##Z', '##q', '##q', '##G', '##Q']
O -100 ['R', '##T']
O -100 ['@']
O -100 ['Doctor', '##Z', '##en']
O -100 [':']
B-group -100 ['G', '##eo', '##logists']
O -100 [',']
B-group -100 ['physicist', '##s']
O -100 [',']
B-group -100 ['chemist', '##s']
O -100 [',']
B-group -100 ['anthropologist', '##s']
O -100 [',']
B-group -100 ['genetic', '##ists']
O -100 ['-']
O -

In [0]:
# Convert to Tensors and build dataset
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)

dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

In [0]:
eval_sampler = SequentialSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=32)
preds = None
out_label_ids = None

In [45]:
model.eval()
preds = []
out_label_ids = []
for batch in tqdm(eval_dataloader, desc="Evaluating"):
    batch = tuple(t.to("cuda") for t in batch)

    with torch.no_grad():
        inputs = {"input_ids": batch[0], "attention_mask": batch[1]}
        inputs["token_type_ids"] = (
            batch[2]
        )
        outputs = model(**inputs)
        last_hidden_layer = outputs[0]
        # if preds is None:
        #     preds = last_hidden_layer.detach().cpu().numpy()
        #     out_label_ids = batch[3].detach().cpu().numpy()
        # else:
        #     preds = np.append(preds, last_hidden_layer.detach().cpu().numpy(), axis=0)
        #     out_label_ids = np.append(out_label_ids, batch[3].detach().cpu().numpy(), axis=0)
        #     print(last_hidden_layer.shape)
        #     print(batch[3].shape)
        preds.append(last_hidden_layer.detach().cpu().numpy())
        out_label_ids.append(batch[3].detach().cpu().numpy())
        # preds[ite*32:(ite+1)*32,:,:] = last_hidden_layer.detach().cpu().numpy()
        # out_label_ids[ite*32:(ite+1)*32,:,:] = out_label_ids.detach().cpu().numpy()
        # ite += 1

In [46]:
out_label_ids[-1].shape

(7, 128)

In [0]:
out_label_list = [[] for _ in range(len(out_label_ids) * 32)]
preds_list = [[] for _ in range(len(out_label_ids) * 32)]

label_map = {i: label for i, label in enumerate(labels)}
        
for i in range(len(out_label_ids)):
    for b in range(out_label_ids[i].shape[0]):
        for j in range(out_label_ids[0].shape[1]):
            if out_label_ids[i][b,j] != pad_token_label_id:
                out_label_list[i*32+b].append(out_label_ids[i][b,j])
                preds_list[i*32+b].append(preds[i][b,j])

In [48]:
out_label_list

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  0,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  4,
  4,
  0,
  0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 3, 2, 0],

`out_label_list` is of shape (#number of sentences, #tokens in this sentence, 1)

`preds_list` is of shape (#number of sentences, #tokens in this sentence, 1024 hidden representations)

For the sake of simplicity, we'll consider that an observation / a sample is a token, not a sentence (the contextual meaning of each token is already taken into account in its representation), so we need to "flatten" both lists so that they're of shape (#tokens, 1)

In [0]:
import itertools

In [0]:
true_labels = np.array(list(itertools.chain.from_iterable(out_label_list)))

In [0]:
flat_list = []
for sentence in preds_list:
    for token in sentence:
        flat_list.append(token)
        
representation = np.array(flat_list)

In [52]:
true_labels.shape

(23364,)

In [53]:
representation.shape

(23364, 1024)

In [0]:
np.save(root+"true_labels.test.npy", true_labels)

In [0]:
np.save(root+"representation.test.npy", representation)

### Loading for example

In [0]:
true_labels = np.load(root+"true_labels.train.npy")

In [59]:
true_labels

array([6, 0, 2, ..., 0, 6, 0])

In [0]:
representation = np.load(root+"representation.train.npy")

In [61]:
representation

array([[-0.5649644 , -0.5623058 , -0.31555155, ..., -0.14965951,
         0.26727486, -0.00221397],
       [-0.3143764 ,  1.3328764 ,  0.9179683 , ...,  0.956362  ,
         0.3026606 , -0.38435155],
       [-0.8777567 ,  0.27505475,  0.41846794, ...,  0.05370105,
         0.08224512,  0.48961633],
       ...,
       [-0.15607439,  0.28895307,  0.45071846, ...,  0.34947208,
         0.13088545,  0.41363466],
       [-0.14588998,  0.46420833, -0.12062413, ..., -0.77222836,
        -0.07450087,  0.7521917 ],
       [-0.3973907 ,  0.15462546, -0.00154371, ...,  0.13841842,
         0.27207437,  0.56779915]], dtype=float32)

### Shrinking attempt

In [0]:
# from scipy import sparse

In [0]:
# representation_sparse = sparse.csr_matrix(representation)

In [0]:
# sparse.save_npz("representation.train.npz", representation_sparse)

Not successfull!

### Loading and converting to CSV: files get too big, you'll have to do this yourself and sample!

In [0]:
from random import sample

In [0]:
representation = np.load("representation.train.npy")
# SAMPLE
int_train = sample(range(representation.shape[0]), 10000)
np.savetxt("representation.train.csv", representation[int_train,:], delimiter=",")
true_labels = np.load("true_labels.train.npy")
np.savetxt("true_labels.train.csv", true_labels[int_train], delimiter=",", fmt='%s')

In [0]:
representation = np.load("representation.testa.npy")
# SAMPLE
int_train = sample(range(representation.shape[0]), 2000)
np.savetxt("representation.testa.csv", representation[int_train,:], delimiter=",")
true_labels = np.load("true_labels.testa.npy")
np.savetxt("true_labels.testa.csv", true_labels[int_train], delimiter=",", fmt='%s')

In [0]:
representation = np.load("representation.testb.npy")
# SAMPLE
int_train = sample(range(representation.shape[0]), 2000)
np.savetxt("representation.testb.csv", representation[int_train,:], delimiter=",")
true_labels = np.load("true_labels.testb.npy")
np.savetxt("true_labels.testb.csv", true_labels[int_train], delimiter=",", fmt='%s')